In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Metabolite_concentrations_diagnosis.csv", sep=";", decimal=",", index_col=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 90
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   METABOLITE 0   91 non-null     float64
 1   METABOLITE 1   91 non-null     float64
 2   METABOLITE 2   91 non-null     float64
 3   METABOLITE 3   91 non-null     float64
 4   METABOLITE 4   91 non-null     float64
 5   METABOLITE 5   91 non-null     float64
 6   METABOLITE 6   91 non-null     float64
 7   METABOLITE 7   91 non-null     float64
 8   METABOLITE 8   91 non-null     float64
 9   METABOLITE 9   91 non-null     float64
 10  METABOLITE 10  91 non-null     float64
 11  METABOLITE 11  91 non-null     float64
 12  METABOLITE 12  91 non-null     float64
 13  METABOLITE 13  91 non-null     float64
 14  METABOLITE 14  91 non-null     float64
 15  TYPE           91 non-null     object 
dtypes: float64(15), object(1)
memory usage: 12.1+ KB


In [3]:
df.head()

,METABOLITE 0,METABOLITE 1,METABOLITE 2,METABOLITE 3,METABOLITE 4,METABOLITE 5,METABOLITE 6,METABOLITE 7,METABOLITE 8,METABOLITE 9,METABOLITE 10,METABOLITE 11,METABOLITE 12,METABOLITE 13,METABOLITE 14,TYPE
0,0.322695,0.575024,0.742645,0.635958,0.544804,0.785336,0.885756,0.577938,0.123017,0.701689,0.731254,0.846980,3.551957,3.742832,1.719949,MENINGIOMA
1,0.457408,0.764700,0.444451,0.451300,0.388213,0.401597,0.382574,0.407632,0.715072,1.216084,1.204490,2.329079,3.179366,3.117351,1.252479,MENINGIOMA
2,1.452870,1.152910,0.647735,0.622494,0.648020,0.826787,0.797439,0.695269,0.850328,0.910488,0.914841,1.805095,2.156737,2.152553,1.081571,MENINGIOMA
3,0.157588,0.314415,0.251613,0.186498,0.100556,0.287618,0.334349,0.213802,0.305852,0.754096,0.627211,1.566407,4.274654,4.210731,1.128131,MENINGIOMA
4,0.368681,0.206993,0.211521,0.034699,0.164187,0.364590,0.340892,0.110287,0.260206,0.531801,0.496068,0.654805,4.070067,4.324002,1.350580,MENINGIOMA


In [4]:
X = df.drop("TYPE", axis=1)
Y = pd.get_dummies(df["TYPE"], columns=["TYPE"])
Y.head()

,ASTROCYTOMA,GLIOBLASTOMA,MENINGIOMA
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
import ydata_profiling as ydp
# generate profiling report and save it
report = ydp.ProfileReport(pd.concat([X_train, y_train], axis=1))
report.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset contains 91 examples described by 16 features – 15 metabolites and type of cancer. The data were divided to the training and test set. The training set will be described further.

There are 63 examples in this set. Each example has one-hot encoded predictive feature – type of cancer. Classes are unbalanced. Meningtoma appears 35 times, astrocytoma – 15 and glioblastoma – 13. It's inversely proportional to the malignancy of the tumor. Therefore, it is crucial to detect the minority classes correctly. 
The metabolites ranges vary, but generally it can be said that there are real numbers mostly greater than 0 and always under 4,5. Distribution of values mostly aren't normal, maybe with except of metabolite 9 (but its a bit skewed) and 11. Some metabolites are highly correlated. Eg. metabolite 12 and 13 are highly correlated and also correlate negatively with all metabolites from 0 to 10. Simmilar situation occurs with metabolites 11 and 14. Moreover metabolite 2 quite highly correlates with 3 and 5 with 6.

In [6]:
import torch
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

X_train, X_test, y_train, y_test = torch.tensor(X_train.values), torch.tensor(X_test.values), torch.tensor(y_train.values), torch.tensor(y_test.values)
X_train = X_train.to(torch.float32)
y_train = y_train.to(torch.float32)
X_test = X_test.to(torch.float32)
y_test = y_test.to(torch.float32)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

class MojaSiec(nn.Module):
    def __init__(self):
        super(MojaSiec, self).__init__()
        self.fc1 = nn.Linear(15, 64)  # Warstwa w pełni połączona z 14 wejściami i 64 neuronami
        self.fc2 = nn.Linear(64, 32)  # Warstwa w pełni połączona z 64 neuronami i 32 neuronami
        self.fc3 = nn.Linear(32, 3)   # Warstwa w pełni połączona z 32 neuronami i 3 wyjściami

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Funkcja aktywacji ReLU po pierwszej warstwie
        x = torch.relu(self.fc2(x))  # Funkcja aktywacji ReLU po drugiej warstwie
        x = self.fc3(x)             # Brak funkcji aktywacji po ostatniej warstwie
        return x

# Inicjalizacja modelu
model = MojaSiec()

# Definicja funkcji straty i optymalizatora
criterion = nn.MSELoss()  # Wybierz odpowiednią funkcję straty
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Wybierz odpowiedni optymalizator i współczynnik uczenia

# Uczenie modelu
epochs = 100  # Liczba epok uczenia
for epoch in range(epochs):
    # Zerowanie gradientów
    optimizer.zero_grad()

    # Przewidywanie na podstawie danych treningowych
    outputs = model(X_train)

    # Obliczenie funkcji straty
    loss = criterion(outputs, y_train)

    # Propagacja wsteczna
    loss.backward()

    # Aktualizacja wag
    optimizer.step()

    if epoch % 10 == 0:
        # Wypisywanie bieżącego stanu uczenia
        print(f'Epoka {epoch+1}/{epochs}, Strata: {loss.item()}')
    
        # Walidacja na zbiorze testowym
        model.eval()  # Ustawienie modelu w tryb ewaluacji (wyłączenie Dropout itp.)

        # Przewidywanie na podstawie danych testowych
        outputs_test = model(X_test)

        # Obliczenie funkcji straty na danych testowych
        loss_test = criterion(outputs_test, y_test)

        # Wypisywanie bieżącego stanu walidacji
        print(f'Strata walidacyjna: {loss_test.item()}')



Epoka 1/100, Strata: 0.3105586767196655
Strata walidacyjna: 0.3054589629173279
Epoka 11/100, Strata: 0.23558780550956726
Strata walidacyjna: 0.22918841242790222
Epoka 21/100, Strata: 0.19713923335075378
Strata walidacyjna: 0.19138677418231964
Epoka 31/100, Strata: 0.18677353858947754
Strata walidacyjna: 0.18139466643333435
Epoka 41/100, Strata: 0.17160911858081818
Strata walidacyjna: 0.16837598383426666
Epoka 51/100, Strata: 0.1562720090150833
Strata walidacyjna: 0.1622355580329895
Epoka 61/100, Strata: 0.14040672779083252
Strata walidacyjna: 0.15643250942230225
Epoka 71/100, Strata: 0.12811188399791718
Strata walidacyjna: 0.1521192342042923
Epoka 81/100, Strata: 0.12158579379320145
Strata walidacyjna: 0.1526457965373993
Epoka 91/100, Strata: 0.11643164604902267
Strata walidacyjna: 0.15137751400470734


In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)  # Ustawiłem dwa ukryte warstwy o rozmiarach 64 i 32, można dostosować

model.fit(X_train, y_train)  # Uczenie modelu na danych treningowych

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Dokładność klasyfikacji: {accuracy * 100:.2f}%')



Dokładność klasyfikacji: 64.29%


/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

model = MLPClassifier(max_iter=1000, random_state=42)

# Definicja przestrzeni parametrów do przeszukania
param_grid = {
    'hidden_layer_sizes': [(32, 32)],  
            "activation": [ "tanh", "relu"],  
}

grid_search = GridSearchCV(model, param_grid, cv=5)  # cv to liczba podziałów dla walidacji krzyżowej

grid_search.fit(X_train, y_train)

print("Najlepsze parametry znalezione w Grid Search:")
print(grid_search.best_params_)

y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Dokładność klasyfikacji na danych testowych: {accuracy * 100:.2f}%')


/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anna/.local/lib/python3.10/site-packages/sklea

Najlepsze parametry znalezione w Grid Search:
{'activation': 'tanh', 'hidden_layer_sizes': (32, 32)}
Dokładność klasyfikacji na danych testowych: 64.29%


/home/anna/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
